# 5. Обработка числовых и категориальных признаков

## 5.1. Масштабирование признаков

In [ ]:
import numpy as np
from sklearn import preprocessing

# Создать признак
X_data = np.array([[-1.0], [-1.5], [2.0], [3.5], [8.0],
                   [5.0], [-2.0], [2.5], [3.5], [6.0]])

# Создание экземпляра класса MinMaxScaler(), диапазон {0,1}
minmax_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))

# Масштабирование признака
X_data_scaled = minmax_scaler.fit_transform(X_data)

# Данные до и после масштабирования
print('До масштабирования:   ', X_data[:5].T)
print('После масштабирования:',  X_data_scaled[:5].T)

# Min и Max значения до и после масштабирования
print('min:', X_data.min(), 'max:', X_data.max())
print('min:', X_data_scaled.min(), 'max:', X_data_scaled.max())

In [ ]:
# Масштабирование данных, имеющих сильные выбросы
# Создать признак
X_data = np.array([[-100.0], [5.0], [2.0], [-3.5], [5.0],
                   [8.0], [-20.0], [3.0], [3.5], [60.0]])

# Создание экземпляра класса RobustScaler()
robust_scaler = preprocessing.RobustScaler()

# Масштабирование признака
X_data_robust = robust_scaler.fit_transform(X_data)

# Данные до и после масштабирования
print('До масштабирования:   ', X_data[:5].T)
print('После масштабирования:',  X_data_scaled[:5].T)

## 5.2. Стандартизация признаков

In [ ]:
import numpy as np
from sklearn import preprocessing

# Создать признак
X_data = np.array([[10.0], [-1.5], [2.0], [3.0], [4.0],
                   [3.0], [-5.0], [2.5], [3.5], [2.0]])

# Создание экземпляра класса StandardScaler()
standard = preprocessing.StandardScaler()

# Стандартизация признака признака
X_data_standard = standard.fit_transform(X_data)

# Данные до и после стандартизации
print('До стандартизации:   ', X_data[:5].T)
print('После стандартизации:',  X_data_scaled[:5].T)

# Данные до и после стандартизации
print('Среднее:', round(X_data_standard.mean()))
print('Стандартное отклонение:', X_data_standard.std())

## 5.3. Нормализация данных

In [ ]:
import numpy as np
from sklearn.preprocessing import Normalizer

# Матрица признаков
X_data = np.array([[0.7,  0.5],
                   [1.5,  2.8],
                   [1.5,  20.2],
                   [2.5,  26.4], 
                   [8.2,  6.3]])

# Создание экземпляра класса Normalizer()
# L-квадрат норма (евклидова норма)
# L^2 = sqrt(x1^2 + x2^2 ... xn^2) = 1

normalizer = Normalizer(norm="l2")
X_norm_l2 = normalizer.transform(X_data)

# Преобразовать матрицу признаков
print(X_norm_l2)

# L' норма (манхэтеннская норма)
# L' = x1 + x2 ... xn = 1

normalizer = Normalizer(norm="l1")
X_norm_l1 = normalizer.transform(X_data)

# Преобразовать матрицу признаков
print(X_norm_l1)

## 5.4. Генерация полиномиальных признаков

In [ ]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

# Матрица признаков
X_data = np.array([[0, 1],
                   [2, 3],
                   [4, 5]])

# Создание экземпляра класса PolynomialFeatures
polynom_data = PolynomialFeatures(degree=2, interaction_only=False)

# Создать новую матрицу признаков
polynom_data.fit_transform(X_data)

# out -> [1, x1, x2, x1^2, x1*x2, x2^2]

## 5.5. Преобразование признаков

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import FunctionTransformer

# Матрица признаков
X_data = np.array([[0, 1, 3],
                   [3, 4, 5],
                   [10, 7, 8],
                   [2, 5, 10]])

# Функция для обработки данных
def transform_func(row):
    return row**2

# Создание экземпляра класса FunctionTransformer()
transform_data = FunctionTransformer(transform_func)

# Создать новую матрицу признаков
transform_data.transform(X_data)

In [ ]:
# Создание датафрейма и использование метода apply()
df = pd.DataFrame(X_data)

df_new = df.apply(transform_func)
df_new.rename(columns=lambda x: 'feature_' + str(x), inplace=True)

## 5.6. Анализ выбросов

In [ ]:
# Использование метода describe()
df_new.describe()

In [ ]:
# Вывод 25-го и 75-го перцентиля данных
df_new.describe()['feature_0'][['25%','75%']]

In [ ]:
q1 = df_new.feature_0.quantile(0.25)  # 25-й перцентиль
q3 = df_new.feature_0.quantile(0.75)  # 75-й перцентиль
iqr = q3 - q1                         # межквартильный размах
lower_bound = q1 - (iqr*1.5)          # нижняя граница выбросов
upper_bound = q3 + (iqr*1.5)          # верхняя граница выбросов
 
print('25-й перцентиль: {},'.format(q1),
      '75-й перцентиль: {},'.format(q3),
      "IQR: {}, ".format(iqr),
      "Границы выбросов: [{lb}, {ub}].".format(lb=lower_bound, ub=upper_bound))

In [ ]:
mean_value = df_new['feature_0'].mean()

def del_outliers(row):
    if row > upper_bound or row < lower_bound:
        return mean_value
    else:
        return row
    
df_new['feature_3'] = df_new['feature_0'].apply(del_outliers)

In [ ]:
import seaborn as sns
from scipy import stats

norm_rv = stats.norm(loc=30, scale=5)
samples = np.trunc(norm_rv.rvs(365))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
fig.suptitle('Распределение признака feature_0')
histplot = sns.histplot(x=samples, ax=axes[0])
boxplot = sns.boxplot(x=samples, ax=axes[1])

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
fig.suptitle('Распределение признака feature_3')
histplot = sns.histplot(x=samples, ax=axes[0])
boxplot = sns.boxplot(x=samples, ax=axes[1])

In [ ]:
# Создание признака на основе булева условия
df_new['feature_4'] = np.where(df_new['feature_1'] < 20,  0,  1)

# Логарифмирование признака
df_new['feature_5'] = [np.log(х) for х in df_new['feature_2']]

## 5.7. Дискретизация признаков

In [ ]:
import numpy as np
from sklearn.preprocessing import Binarizer

# Создать признак
X_data = np.array([[1], [10], [100], [1000]])

# Создание экземпляра класса Binarizer()
binarizer = Binarizer(50)                # порого бинаризации = 50

# Бинаризация признака
X_binar = binarizer.fit_transform(X_data)
print(X_binar.T)

In [ ]:
# Разбиение на диапазоны
X_binar_digit = np.digitize(X_data, bins=[1, 10, 100, 1000])
print(X_binar_digit.T)

## 5.8. Работа с пропущенными данными

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Создание списка с пропущенными данными
numbers = [[1, 8, np.nan], 
           [2, np.nan, 16], 
           [np.nan, 10, 17],
           [7, np.nan, 18], 
           [5, 12, np.nan]] 
 
header = ['feature_1', 'feature_2', 'feature_3'] 

# Создание датафрейма
df = pd.DataFrame(data = numbers, columns = header)

# Подсчёт пропущенных значений в датафрейме
df.isnull().sum()

In [ ]:
# Визуализация пропусков
plt.figure(figsize=(6,4))
sns.heatmap(df.isna().transpose(),
            cmap="YlGnBu",
            cbar_kws={'label': 'Пропущенные данные'})
plt.show()

In [ ]:
# Замена пропусков

feat_1_mean = df['feature_1'].mean()                  # расчёт среднего
feat_2_median = df['feature_2'].median()              # расчёт медианы
feat_3_mode = df['feature_3'].mode()                  # расчёт моды

df['feature_1'].fillna(feat_1_mean, inplace=True)     # замена пропусков на среднее
df['feature_2'].fillna(feat_2_median, inplace=True)   # замена пропусков на медиану
df['feature_3'].fillna(feat_3_mode[0], inplace=True)  # замена пропусков на моду

In [ ]:
# Импутация признаков
from sklearn.impute import SimpleImputer

X_data = np.array(numbers)

# Создание экземпляра класса SimpleImputer()
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

# Импутация значений
mean_imputed = mean_imputer.fit_transform(X_data)

print("Импутированное значение:",  mean_imputed[0,2])

## 5.9. Кодирование номинальных категориальных признаков

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

# Создание признака
X_data = np.array([['pressure'],
                   ['gas'],
                   ['pressure'],
                   ['temp'],
                   ['pressure']])

# Создание экземпляра класса LabelBinarizer()
one_hot = LabelBinarizer()

# Кодирование признака
one_hot.fit_transform(X_data)

In [ ]:
# Создание мультипризнака
X_data_mult = np.array([['pressure','humidity'],
                        ['gas','pressure', 'temp'],
                        ['pressure','humidity'],
                        ['temp','pressure'],
                        ['pressure','humidity']])

# Создание экземпляра класса MultiLabelBinarizer()
one_hot = MultiLabelBinarizer()

# Кодирование признака
one_hot.fit_transform(X_data_mult)

In [ ]:
# Кодирование признака и создание датафрейма
pd.get_dummies(X_data[:,0])

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder

# Создание словаря
data = {'temp': ['high', 'low', 'high', 'low'], 
        'size': ['small', 'big', 'big', 'medium'], 
        'pressure': ['isobar','isochor','isother','isochor']}

# Создание датафрейма
df = pd.DataFrame(data = data)

# Создание экземпляра класса OrdinalEncoder()
ord_enc = OrdinalEncoder()

# Кодирование признака
df['pressure_ordinal'] = ord_enc.fit_transform(df[['pressure']])

# Создание экземпляра класса LabelEncoder()
lab_enc = LabelEncoder()

# Кодирование признака
df['pressure_label'] = lab_enc.fit_transform(df['pressure'])

# Создание экземпляра класса OneHotEncoder()
ohe_enc = OneHotEncoder()
# Кодирование признака
ohe_res = ohe_enc.fit_transform(df[['pressure']])

ohe_df = pd.DataFrame(ohe_res.toarray(), columns=list(*ohe_enc.categories_))
df = pd.concat([df, ohe_df], axis=1)

# OHE с использованием библиотеки pandas
pd.get_dummies(df, columns=['size','temp'], prefix=['size','temp'])

# Проверка типа данных
type(df['pressure_ordinal'].iloc[2])
type(df['pressure_label'].iloc[2])

In [ ]:
import category_encoders as ce

# Создание экземпляра класса OrdinalEncoder()
ord_enc = ce.OrdinalEncoder()
data_ord = ord_enc.fit_transform(df['size'])
df = pd.concat([df, data_ord], axis=1)

# Создание экземпляра класса OneHotEncoder()
ohe_enc = ce.OneHotEncoder(cols=['pressure'])
data_ohe = ohe_enc.fit_transform(df['pressure'])
df = pd.concat([df, data_ohe], axis=1)

# Создание экземпляра класса BinaryEncoder()
bin_enc = ce.BinaryEncoder(cols=['pressure'])
data_bin = bin_enc.fit_transform(data['pressure'])
df = pd.concat([df, data_bin], axis=1)

## 5.10. Кодирование порядковых категориальных признаков

In [ ]:
# Замена порядковых категориальных признаков методом replace()

import pandas as pd 

# Создание словаря
data = {'temp': ['high', 'low', 'high', 'middle'], 
        'size': ['small', 'big', 'big', 'medium'], 
        'pressure': ['isobar','isochor','isother','isochor']}

# Создание датафрейма
df = pd.DataFrame(data = data)

scale_rep =  {'small': 1, 'medium': 2, 'big': 3}
df['size'] = df['size'].replace(scale_rep)

# Замена признаков с использованием вложенных словарей
scale_rdic =  {'temp': {'low': 1, 'middle': 2, 'high': 3}}
df = df.replace(scale_rdic)

# Замена порядковых категориальных признаков методом map()
scale_map =  {'isochor': 1, 'isother': 2, 'isobar': 3}
df['pressure'] = df['pressure'].map(scale_map)

## 5.11. Кодирование словарей признаков

In [ ]:
# Создание массива категориальных признаков на основе словарей

from sklearn.feature_extraction import DictVectorizer

# Создание словаря с данными
data_dict =  [{'high': 2,  'low':  5},
              {'high': 4,  'low':  3},
              {'high': 3,  'middle': 2},
              {'high': 2,  'middle': 4}]
               
# Создание экземпляра класса DictVectorizer()
dictvec = DictVectorizer(sparse=False)

# Кодировка признаков
features = dictvec.fit_transform(data_dict)

# Матрица признаков и названия признаков
feature_names = dictvec.get_feature_names()

# если sparse=True, то features.toarray()
features, feature_names

## 5.12. Категориальные переменные

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Создание датафрейма на основе генерации случайных чисел
df = pd.DataFrame({'value': np.random.randint(0, 100, 1000)})

# Создание меток категориий
labels = ['{0} - {1}'.format(i, i + 9) for i in range(0, 100, 10)]
# Создание признака категорий
df['value_group'] = pd.cut(df.value, range(0, 105, 10), right=False, labels=labels)

sns.countplot(x='value_group', data=df)
plt.show()